# 🌐 깊이 계산 및 3D 재구성 실습

이 노트북에서는 시차 맵으로부터 실제 깊이 정보를 계산하고, 3D 점군을 생성하여 3차원 재구성을 수행합니다.

## 🎯 학습 목표
- 시차 맵을 실제 깊이(거리)로 변환하는 방법 이해
- 카메라 매개변수가 깊이 계산에 미치는 영향 분석
- 3D 점군 생성 및 시각화
- 깊이 맵 품질 평가 및 분석
- 실제 거리 측정 시스템 구축

## 📦 라이브러리 import 및 설정

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
from matplotlib.patches import Rectangle
import warnings
warnings.filterwarnings('ignore')

# 시각화 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (15, 10)

print("✅ 라이브러리 import 완료!")
print("🌐 3D 재구성 준비 완료!")
print("📐 깊이 계산 공식: Z = (f × b) / d")
print("   - f: 초점거리 (focal length)")
print("   - b: 베이스라인 (baseline)")
print("   - d: 시차 (disparity)")
print("   - Z: 깊이 (depth)")

## 🏗️ DepthCalculator 클래스 구현

In [ ]:
class DepthCalculator:
    """
    깊이 계산 및 3D 재구성을 위한 클래스
    """
    
    def __init__(self, focal_length, baseline, image_width, image_height, 
                 principal_point_x=None, principal_point_y=None):
        """
        깊이 계산기 초기화
        
        Parameters:
        - focal_length: 초점거리 (픽셀 단위)
        - baseline: 베이스라인 (미터 단위)
        - image_width, image_height: 영상 크기
        - principal_point_x, y: 주점 좌표 (기본값: 영상 중심)
        """
        self.focal_length = focal_length
        self.baseline = baseline
        self.image_width = image_width
        self.image_height = image_height
        
        # 주점 설정 (카메라 내부 매개변수)
        self.cx = principal_point_x if principal_point_x is not None else image_width / 2
        self.cy = principal_point_y if principal_point_y is not None else image_height / 2
        
        # 카메라 내부 매개변수 행렬 (K matrix)
        self.K = np.array([
            [focal_length, 0, self.cx],
            [0, focal_length, self.cy],
            [0, 0, 1]
        ])
        
        print(f"🔧 DepthCalculator 초기화 완료")
        print(f"   📏 초점거리: {focal_length:.1f} pixels")
        print(f"   📐 베이스라인: {baseline:.3f} m ({baseline*100:.1f} cm)")
        print(f"   📺 영상 크기: {image_width} x {image_height}")
        print(f"   🎯 주점: ({self.cx:.1f}, {self.cy:.1f})")
        
        # 이론적 측정 범위 계산
        self._calculate_measurement_range()
    
    def _calculate_measurement_range(self):
        """
        이론적 측정 범위 계산
        """
        # 최소 거리 (최대 시차 1픽셀일 때)
        max_disparity = min(self.image_width // 4, 100)  # 실용적 최대 시차
        min_distance = (self.focal_length * self.baseline) / max_disparity
        
        # 최대 거리 (최소 시차 0.1픽셀일 때)
        min_disparity = 0.1
        max_distance = (self.focal_length * self.baseline) / min_disparity
        
        print(f"   📊 이론적 측정 범위: {min_distance:.2f}m ~ {max_distance:.1f}m")
        print(f"   ⚠️ 실제 정확도는 시차가 클수록 (가까운 거리일수록) 높음")
    
    def disparity_to_depth(self, disparity_map, min_disparity=0.1, max_depth=100.0):
        """
        시차 맵을 깊이 맵으로 변환
        
        Parameters:
        - disparity_map: 시차 맵
        - min_disparity: 최소 시차 (0으로 나누기 방지)
        - max_depth: 최대 깊이 제한
        
        Returns:
        - depth_map: 깊이 맵 (미터 단위)
        """
        print("🔄 시차 맵을 깊이 맵으로 변환 중...")
        
        # 유효한 시차 마스크 생성
        valid_mask = disparity_map > min_disparity
        
        # 깊이 맵 초기화
        depth_map = np.full_like(disparity_map, np.inf, dtype=np.float32)
        
        # 유효한 영역에서만 깊이 계산
        depth_map[valid_mask] = (self.focal_length * self.baseline) / disparity_map[valid_mask]
        
        # 최대 깊이 제한 적용
        depth_map = np.clip(depth_map, 0, max_depth)
        
        # 무효한 영역은 0으로 설정
        depth_map[~valid_mask] = 0
        
        print(f"✅ 깊이 변환 완료!")
        
        return depth_map
    
    def create_3d_points(self, disparity_map, left_image=None, max_depth=50.0):
        """
        시차 맵으로부터 3D 점군 생성
        
        Parameters:
        - disparity_map: 시차 맵
        - left_image: 좌측 영상 (텍스처용, 선택사항)
        - max_depth: 최대 깊이
        
        Returns:
        - points_3d: 3D 좌표 배열 (N x 3)
        - colors: 색상 정보 (N x 3) 또는 (N,)
        """
        print("🌐 3D 점군 생성 중...")
        
        # 깊이 맵 계산
        depth_map = self.disparity_to_depth(disparity_map, max_depth=max_depth)
        
        # 유효한 깊이 마스크
        valid_mask = (depth_map > 0) & (depth_map < max_depth)
        
        # 픽셀 좌표 생성
        y_coords, x_coords = np.mgrid[0:self.image_height, 0:self.image_width]
        
        # 유효한 점들만 선택
        valid_x = x_coords[valid_mask]
        valid_y = y_coords[valid_mask]
        valid_depth = depth_map[valid_mask]
        
        # 3D 좌표 계산 (카메라 좌표계)
        # X = (x - cx) * Z / f
        # Y = (y - cy) * Z / f
        # Z = depth
        points_3d = np.zeros((len(valid_x), 3), dtype=np.float32)
        points_3d[:, 0] = (valid_x - self.cx) * valid_depth / self.focal_length  # X
        points_3d[:, 1] = (valid_y - self.cy) * valid_depth / self.focal_length  # Y
        points_3d[:, 2] = valid_depth  # Z
        
        # 색상 정보 추출
        if left_image is not None:
            if len(left_image.shape) == 3:
                # 컬러 영상
                colors = left_image[valid_mask]
            else:
                # 그레이스케일 영상
                gray_values = left_image[valid_mask]
                colors = np.stack([gray_values, gray_values, gray_values], axis=1)
        else:
            # 깊이 기반 색상 생성
            normalized_depth = (valid_depth - valid_depth.min()) / (valid_depth.max() - valid_depth.min())
            colors = plt.cm.viridis(normalized_depth)[:, :3] * 255
        
        print(f"✅ 3D 점군 생성 완료! 총 {len(points_3d):,}개 점")
        
        return points_3d, colors

# 예시 카메라 매개변수로 객체 생성
depth_calc = DepthCalculator(
    focal_length=600,  # 픽셀
    baseline=0.12,     # 12cm
    image_width=640,
    image_height=480
)

## 🎨 현실적인 스테레오 데이터 생성

실제 깊이 정보가 포함된 테스트 데이터를 생성합니다.

In [ ]:
def create_realistic_stereo_scene():
    """
    실제적인 스테레오 장면 생성
    다양한 거리의 객체들과 현실적인 시차 패턴 포함
    """
    height, width = 480, 640
    
    # 좌측 영상 생성
    left_img = np.zeros((height, width), dtype=np.uint8)
    
    # 배경 그라디언트 (멀리 있는 벽)
    for i in range(height):
        left_img[i, :] = int(80 + (i / height) * 50)
    
    # 실제 거리 정보와 함께 객체들 배치
    objects_info = [
        {'name': 'Near Box', 'distance': 1.0, 'disparity': 72, 'rect': (150, 200, 100, 80), 'intensity': 220},
        {'name': 'Middle Sphere', 'distance': 2.0, 'disparity': 36, 'center': (450, 150), 'radius': 50, 'intensity': 180},
        {'name': 'Far Block', 'distance': 3.0, 'disparity': 24, 'rect': (300, 350, 120, 60), 'intensity': 160},
        {'name': 'Very Far Wall', 'distance': 6.0, 'disparity': 12, 'rect': (50, 100, 200, 150), 'intensity': 140}
    ]
    
    # 객체들 그리기
    for obj in objects_info:
        if 'rect' in obj:
            x, y, w, h = obj['rect']
            left_img[y:y+h, x:x+w] = obj['intensity']
            # 텍스트 추가
            cv2.putText(left_img, f"{obj['distance']:.1f}m", (x+5, y+20), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, 255, 1)
        elif 'center' in obj:
            center = obj['center']
            radius = obj['radius']
            cv2.circle(left_img, center, radius, obj['intensity'], -1)
            cv2.putText(left_img, f"{obj['distance']:.1f}m", (center[0]-20, center[1]), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, 255, 1)
    
    # 텍스처 패턴 추가
    for i in range(0, width, 40):
        cv2.line(left_img, (i, 0), (i, height), 100, 1)
    for i in range(0, height, 40):
        cv2.line(left_img, (0, i), (width, i), 100, 1)
    
    # 우측 영상 생성 (시차 적용)
    right_img = left_img.copy()
    
    # 배경 다시 생성
    right_img.fill(0)
    for i in range(height):
        right_img[i, :] = int(80 + (i / height) * 50)
    
    # 텍스처 패턴 (약간 이동)
    for i in range(0, width, 40):
        cv2.line(right_img, (i-2, 0), (i-2, height), 100, 1)
    for i in range(0, height, 40):
        cv2.line(right_img, (0, i), (width, i), 100, 1)
    
    # 객체들을 시차만큼 이동시켜 배치
    for obj in objects_info:
        disparity = obj['disparity']
        if 'rect' in obj:
            x, y, w, h = obj['rect']
            new_x = max(0, x - disparity)
            if new_x + w < width:
                right_img[y:y+h, new_x:new_x+w] = obj['intensity']
                cv2.putText(right_img, f"{obj['distance']:.1f}m", (new_x+5, y+20), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, 255, 1)
        elif 'center' in obj:
            center = obj['center']
            new_center = (max(obj['radius'], center[0] - disparity), center[1])
            radius = obj['radius']
            cv2.circle(right_img, new_center, radius, obj['intensity'], -1)
            cv2.putText(right_img, f"{obj['distance']:.1f}m", (new_center[0]-20, new_center[1]), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, 255, 1)
    
    # 현실적인 노이즈 추가
    noise_left = np.random.normal(0, 5, (height, width))
    noise_right = np.random.normal(0, 5, (height, width))
    
    left_img = np.clip(left_img.astype(np.float32) + noise_left, 0, 255).astype(np.uint8)
    right_img = np.clip(right_img.astype(np.float32) + noise_right, 0, 255).astype(np.uint8)
    
    return left_img, right_img, objects_info

# 현실적인 스테레오 장면 생성
left_image, right_image, scene_objects = create_realistic_stereo_scene()

# 영상 시각화
plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
plt.imshow(left_image, cmap='gray')
plt.title('Realistic Left Image\n(실제적인 좌측 영상)', fontsize=14)
plt.axis('off')

# 객체 정보 표시
for obj in scene_objects:
    if 'rect' in obj:
        x, y, w, h = obj['rect']
        rect = Rectangle((x, y), w, h, linewidth=2, edgecolor='red', facecolor='none')
        plt.gca().add_patch(rect)

plt.subplot(1, 2, 2)
plt.imshow(right_image, cmap='gray')
plt.title('Realistic Right Image\n(실제적인 우측 영상)', fontsize=14)
plt.axis('off')

plt.tight_layout()
plt.show()

print("✅ 현실적인 스테레오 장면 생성 완료!")
print(f"📐 영상 크기: {left_image.shape}")
print("🎯 포함된 객체들:")
for obj in scene_objects:
    print(f"   - {obj['name']}: {obj['distance']:.1f}m (시차 {obj['disparity']}픽셀)")

## 🔧 스테레오 매칭으로 시차 맵 생성

In [ ]:
# OpenCV StereoSGBM으로 고품질 시차 맵 생성
def compute_high_quality_disparity(left_img, right_img):
    """
    고품질 시차 맵 계산
    """
    print("🔄 고품질 시차 맵 계산 중...")
    
    # StereoSGBM 매개변수 최적화
    stereo = cv2.StereoSGBM_create(
        minDisparity=0,
        numDisparities=96,  # 더 넓은 시차 범위
        blockSize=5,
        P1=8 * 3 * 5**2,
        P2=32 * 3 * 5**2,
        disp12MaxDiff=1,
        uniquenessRatio=10,
        speckleWindowSize=100,
        speckleRange=32,
        preFilterCap=63,
        mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
    )
    
    # 시차 계산
    disparity = stereo.compute(left_img, right_img)
    
    # 16배수로 반환되므로 실제 값으로 변환
    disparity_real = disparity.astype(np.float32) / 16.0
    
    # 음수 값 제거
    disparity_real[disparity_real < 0] = 0
    
    print("✅ 시차 맵 계산 완료!")
    
    return disparity_real

# 시차 맵 계산
disparity_map = compute_high_quality_disparity(left_image, right_image)

# 시차 맵 시각화
plt.figure(figsize=(18, 6))

plt.subplot(1, 3, 1)
plt.imshow(left_image, cmap='gray')
plt.title('Left Image', fontsize=14)
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(right_image, cmap='gray')
plt.title('Right Image', fontsize=14)
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(disparity_map, cmap='plasma')
plt.title('Disparity Map\n(시차 맵)', fontsize=14)
plt.colorbar(label='Disparity (pixels)')
plt.axis('off')

plt.tight_layout()
plt.show()

# 시차 맵 통계
valid_disparity = disparity_map[disparity_map > 0]
print(f"\n📊 시차 맵 통계:")
if len(valid_disparity) > 0:
    print(f"   - 최소 시차: {valid_disparity.min():.2f} 픽셀")
    print(f"   - 최대 시차: {valid_disparity.max():.2f} 픽셀")
    print(f"   - 평균 시차: {valid_disparity.mean():.2f} 픽셀")
    print(f"   - 유효 픽셀: {len(valid_disparity):,}개 ({len(valid_disparity)/disparity_map.size*100:.1f}%)")

## 🌊 깊이 맵 생성 및 분석

In [ ]:
# 깊이 맵 계산
depth_map = depth_calc.disparity_to_depth(disparity_map, min_disparity=0.5, max_depth=20.0)

# 깊이 맵 분석 함수
def analyze_depth_map(depth_map, disparity_map, scene_objects):
    """
    깊이 맵 상세 분석
    """
    print("📊 깊이 맵 분석 중...")
    
    valid_mask = depth_map > 0
    valid_depths = depth_map[valid_mask]
    
    if len(valid_depths) == 0:
        print("❌ 유효한 깊이 데이터가 없습니다.")
        return {}
    
    stats = {
        'min_depth': np.min(valid_depths),
        'max_depth': np.max(valid_depths),
        'mean_depth': np.mean(valid_depths),
        'median_depth': np.median(valid_depths),
        'std_depth': np.std(valid_depths),
        'valid_pixels': len(valid_depths),
        'total_pixels': depth_map.size,
        'coverage': len(valid_depths) / depth_map.size
    }
    
    print(f"\n📏 깊이 통계:")
    print(f"   - 최소 거리: {stats['min_depth']:.2f} m")
    print(f"   - 최대 거리: {stats['max_depth']:.2f} m")
    print(f"   - 평균 거리: {stats['mean_depth']:.2f} m")
    print(f"   - 중간값: {stats['median_depth']:.2f} m")
    print(f"   - 표준편차: {stats['std_depth']:.2f} m")
    print(f"   - 커버리지: {stats['coverage']*100:.1f}%")
    
    # 실제 객체와 측정값 비교
    print(f"\n🎯 실제 거리 vs 측정 거리:")
    for obj in scene_objects:
        actual_distance = obj['distance']
        if 'rect' in obj:
            x, y, w, h = obj['rect']
            # 객체 중심 영역의 평균 깊이
            center_x, center_y = x + w//2, y + h//2
            region = depth_map[center_y-10:center_y+10, center_x-10:center_x+10]
            valid_region = region[region > 0]
            if len(valid_region) > 0:
                measured_distance = np.mean(valid_region)
                error = abs(measured_distance - actual_distance)
                error_percent = (error / actual_distance) * 100
                print(f"   - {obj['name']}: 실제 {actual_distance:.1f}m, 측정 {measured_distance:.2f}m, 오차 {error_percent:.1f}%")
    
    return stats

# 깊이 맵 분석
depth_stats = analyze_depth_map(depth_map, disparity_map, scene_objects)

# 깊이 맵 시각화
plt.figure(figsize=(20, 12))

# 원본 이미지와 시차 맵
plt.subplot(2, 3, 1)
plt.imshow(left_image, cmap='gray')
plt.title('Original Image', fontsize=14)
plt.axis('off')

plt.subplot(2, 3, 2)
plt.imshow(disparity_map, cmap='plasma')
plt.title('Disparity Map\n(시차 맵)', fontsize=14)
plt.colorbar(label='Disparity (pixels)')
plt.axis('off')

# 깊이 맵
plt.subplot(2, 3, 3)
depth_display = np.where(depth_map > 0, depth_map, np.nan)
plt.imshow(depth_display, cmap='viridis_r')  # 가까운 것이 밝게
plt.title('Depth Map\n(깊이 맵)', fontsize=14)
plt.colorbar(label='Distance (meters)')
plt.axis('off')

# 깊이 히스토그램
plt.subplot(2, 3, 4)
valid_depths = depth_map[depth_map > 0]
if len(valid_depths) > 0:
    plt.hist(valid_depths, bins=50, alpha=0.7, edgecolor='black')
    plt.axvline(np.mean(valid_depths), color='red', linestyle='--', label=f'Mean: {np.mean(valid_depths):.2f}m')
    plt.axvline(np.median(valid_depths), color='green', linestyle='--', label=f'Median: {np.median(valid_depths):.2f}m')
plt.xlabel('Distance (meters)')
plt.ylabel('Frequency')
plt.title('Depth Distribution', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)

# 시차-깊이 관계
plt.subplot(2, 3, 5)
valid_mask = (disparity_map > 0) & (depth_map > 0)
if np.sum(valid_mask) > 0:
    sample_size = min(5000, np.sum(valid_mask))
    indices = np.random.choice(np.sum(valid_mask), sample_size, replace=False)
    
    valid_disp = disparity_map[valid_mask][indices]
    valid_depth = depth_map[valid_mask][indices]
    
    plt.scatter(valid_disp, valid_depth, alpha=0.5, s=1)
    
    # 이론적 곡선 추가
    disp_range = np.linspace(valid_disp.min(), valid_disp.max(), 100)
    theoretical_depth = (depth_calc.focal_length * depth_calc.baseline) / disp_range
    plt.plot(disp_range, theoretical_depth, 'r-', linewidth=2, label='Theoretical')

plt.xlabel('Disparity (pixels)')
plt.ylabel('Depth (meters)')
plt.title('Disparity vs Depth', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)

# 오차 분석
plt.subplot(2, 3, 6)
# 거리별 측정 정확도
distance_ranges = [(0, 1), (1, 2), (2, 3), (3, 5), (5, 10)]
range_labels = ['0-1m', '1-2m', '2-3m', '3-5m', '5-10m']
accuracies = []

for min_d, max_d in distance_ranges:
    mask = (depth_map >= min_d) & (depth_map < max_d)
    if np.sum(mask) > 0:
        # 상대적 정확도 (표준편차/평균)
        depths_in_range = depth_map[mask]
        relative_accuracy = np.std(depths_in_range) / np.mean(depths_in_range)
        accuracies.append(relative_accuracy)
    else:
        accuracies.append(0)

bars = plt.bar(range_labels, accuracies, alpha=0.7)
plt.ylabel('Relative Error (std/mean)')
plt.title('Accuracy by Distance', fontsize=14)
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 깊이 맵 해석:")
print("   🟣 보라색/어두운 영역: 가까운 물체")
print("   🟡 노란색/밝은 영역: 먼 물체")
print("   ⚫ 검은색 영역: 측정 불가능한 영역")
print("   📈 정확도는 가까운 거리에서 높고 먼 거리에서 낮음")

## 🌐 3D 점군 생성 및 시각화

In [ ]:
# 3D 점군 생성
points_3d, colors = depth_calc.create_3d_points(disparity_map, left_image, max_depth=15.0)

def visualize_3d_point_cloud(points_3d, colors, title="3D Point Cloud", max_points=15000):
    """
    3D 점군 시각화
    """
    # 점이 너무 많으면 샘플링
    if len(points_3d) > max_points:
        indices = np.random.choice(len(points_3d), max_points, replace=False)
        points_3d_sample = points_3d[indices]
        colors_sample = colors[indices]
    else:
        points_3d_sample = points_3d
        colors_sample = colors
    
    # 다양한 시점으로 3D 시각화
    fig = plt.figure(figsize=(20, 12))
    
    # 시점 1: 정면
    ax1 = fig.add_subplot(2, 3, 1, projection='3d')
    if len(colors_sample.shape) == 2 and colors_sample.shape[1] == 3:
        ax1.scatter(points_3d_sample[:, 0], points_3d_sample[:, 1], points_3d_sample[:, 2],
                   c=colors_sample/255.0, s=0.5, alpha=0.8)
    else:
        ax1.scatter(points_3d_sample[:, 0], points_3d_sample[:, 1], points_3d_sample[:, 2],
                   c=colors_sample, cmap='gray', s=0.5, alpha=0.8)
    
    ax1.set_xlabel('X (meters)')
    ax1.set_ylabel('Y (meters)')
    ax1.set_zlabel('Z (meters)')
    ax1.set_title('Front View\n(정면 시점)')
    ax1.view_init(elev=0, azim=0)
    
    # 시점 2: 위에서
    ax2 = fig.add_subplot(2, 3, 2, projection='3d')
    if len(colors_sample.shape) == 2 and colors_sample.shape[1] == 3:
        ax2.scatter(points_3d_sample[:, 0], points_3d_sample[:, 1], points_3d_sample[:, 2],
                   c=colors_sample/255.0, s=0.5, alpha=0.8)
    else:
        ax2.scatter(points_3d_sample[:, 0], points_3d_sample[:, 1], points_3d_sample[:, 2],
                   c=colors_sample, cmap='gray', s=0.5, alpha=0.8)
    
    ax2.set_xlabel('X (meters)')
    ax2.set_ylabel('Y (meters)')
    ax2.set_zlabel('Z (meters)')
    ax2.set_title('Top View\n(위에서 시점)')
    ax2.view_init(elev=90, azim=0)
    
    # 시점 3: 비스듬히
    ax3 = fig.add_subplot(2, 3, 3, projection='3d')
    if len(colors_sample.shape) == 2 and colors_sample.shape[1] == 3:
        ax3.scatter(points_3d_sample[:, 0], points_3d_sample[:, 1], points_3d_sample[:, 2],
                   c=colors_sample/255.0, s=0.5, alpha=0.8)
    else:
        ax3.scatter(points_3d_sample[:, 0], points_3d_sample[:, 1], points_3d_sample[:, 2],
                   c=colors_sample, cmap='gray', s=0.5, alpha=0.8)
    
    ax3.set_xlabel('X (meters)')
    ax3.set_ylabel('Y (meters)')
    ax3.set_zlabel('Z (meters)')
    ax3.set_title('Isometric View\n(등각 시점)')
    ax3.view_init(elev=20, azim=45)
    
    # 2D 투영들
    # XY 평면 (위에서 보기)
    ax4 = fig.add_subplot(2, 3, 4)
    scatter = ax4.scatter(points_3d_sample[:, 0], points_3d_sample[:, 1], 
                         c=points_3d_sample[:, 2], cmap='viridis', s=1, alpha=0.6)
    ax4.set_xlabel('X (meters)')
    ax4.set_ylabel('Y (meters)')
    ax4.set_title('XY Projection (Top View)')
    plt.colorbar(scatter, ax=ax4, label='Z (meters)')
    ax4.grid(True, alpha=0.3)
    
    # XZ 평면 (정면에서 보기)
    ax5 = fig.add_subplot(2, 3, 5)
    scatter = ax5.scatter(points_3d_sample[:, 0], points_3d_sample[:, 2], 
                         c=points_3d_sample[:, 1], cmap='plasma', s=1, alpha=0.6)
    ax5.set_xlabel('X (meters)')
    ax5.set_ylabel('Z (meters)')
    ax5.set_title('XZ Projection (Front View)')
    plt.colorbar(scatter, ax=ax5, label='Y (meters)')
    ax5.grid(True, alpha=0.3)
    
    # 3D 통계
    ax6 = fig.add_subplot(2, 3, 6)
    ax6.axis('off')
    
    stats_text = f"""
3D Point Cloud Statistics
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Total Points: {len(points_3d):,}
Displayed Points: {len(points_3d_sample):,}

X Range: {points_3d_sample[:, 0].min():.2f} ~ {points_3d_sample[:, 0].max():.2f} m
Y Range: {points_3d_sample[:, 1].min():.2f} ~ {points_3d_sample[:, 1].max():.2f} m
Z Range: {points_3d_sample[:, 2].min():.2f} ~ {points_3d_sample[:, 2].max():.2f} m

Centroid:
  X: {np.mean(points_3d_sample[:, 0]):.2f} m
  Y: {np.mean(points_3d_sample[:, 1]):.2f} m
  Z: {np.mean(points_3d_sample[:, 2]):.2f} m

Standard Deviation:
  X: {np.std(points_3d_sample[:, 0]):.2f} m
  Y: {np.std(points_3d_sample[:, 1]):.2f} m
  Z: {np.std(points_3d_sample[:, 2]):.2f} m
    """
    
    ax6.text(0.1, 0.9, stats_text, transform=ax6.transAxes, 
             verticalalignment='top', fontfamily='monospace', fontsize=10)
    
    plt.tight_layout()
    plt.show()

# 3D 점군 시각화
visualize_3d_point_cloud(points_3d, colors, "Stereo Vision 3D Reconstruction")

print(f"\n🌐 3D 재구성 완료!")
print(f"   📊 총 점 개수: {len(points_3d):,}개")
print(f"   📏 공간 범위: X({points_3d[:, 0].min():.2f}~{points_3d[:, 0].max():.2f}m), "
      f"Y({points_3d[:, 1].min():.2f}~{points_3d[:, 1].max():.2f}m), "
      f"Z({points_3d[:, 2].min():.2f}~{points_3d[:, 2].max():.2f}m)")
print(f"   🎯 중심점: ({np.mean(points_3d[:, 0]):.2f}, {np.mean(points_3d[:, 1]):.2f}, {np.mean(points_3d[:, 2]):.2f})m")

## 🔬 카메라 매개변수 영향 분석

In [ ]:
def analyze_camera_parameter_effects():
    """
    카메라 매개변수가 깊이 계산에 미치는 영향 분석
    """
    print("🔬 카메라 매개변수 영향 분석 시작...")
    
    # 다양한 매개변수 조합
    parameter_sets = [
        {'focal_length': 400, 'baseline': 0.08, 'name': 'Low F, Small B'},
        {'focal_length': 600, 'baseline': 0.12, 'name': 'Medium F, Medium B'},
        {'focal_length': 800, 'baseline': 0.16, 'name': 'High F, Large B'},
        {'focal_length': 600, 'baseline': 0.20, 'name': 'Medium F, Large B'},
    ]
    
    results = []
    
    # 고정된 시차 값들로 테스트
    test_disparities = np.array([5, 10, 20, 40, 60])
    
    plt.figure(figsize=(20, 12))
    
    for i, params in enumerate(parameter_sets):
        # 각 매개변수로 깊이 계산
        depths = (params['focal_length'] * params['baseline']) / test_disparities
        
        # 정확도 계산 (시차 1픽셀 오차 시)
        depth_errors = []
        for d in test_disparities:
            true_depth = (params['focal_length'] * params['baseline']) / d
            error_depth = (params['focal_length'] * params['baseline']) / (d + 1)  # 1픽셀 오차
            relative_error = abs(error_depth - true_depth) / true_depth * 100
            depth_errors.append(relative_error)
        
        results.append({
            'params': params,
            'depths': depths,
            'errors': depth_errors
        })
        
        # 시차-깊이 관계 그래프
        plt.subplot(2, 2, i+1)
        
        # 연속적인 시차 범위로 부드러운 곡선 그리기
        disp_range = np.linspace(1, 80, 100)
        depth_range = (params['focal_length'] * params['baseline']) / disp_range
        
        plt.plot(disp_range, depth_range, 'b-', linewidth=2, label='Theoretical')
        plt.scatter(test_disparities, depths, color='red', s=50, zorder=5, label='Test Points')
        
        plt.xlabel('Disparity (pixels)')
        plt.ylabel('Depth (meters)')
        plt.title(f"{params['name']}\nf={params['focal_length']}, b={params['baseline']*100:.0f}cm")
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.xlim(0, 80)
        plt.ylim(0, 10)
        
        # 측정 범위 표시
        min_dist = (params['focal_length'] * params['baseline']) / 80
        max_dist = (params['focal_length'] * params['baseline']) / 1
        plt.axhspan(min_dist, min(max_dist, 10), alpha=0.2, color='green')
        
        # 텍스트 정보 추가
        info_text = f"Range: {min_dist:.2f}m - {min(max_dist, 10):.1f}m"
        plt.text(0.05, 0.95, info_text, transform=plt.gca().transAxes, 
                bbox=dict(boxstyle="round", facecolor='wheat', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    # 정확도 비교 그래프
    plt.figure(figsize=(15, 8))
    
    # 거리별 오차 비교
    plt.subplot(1, 2, 1)
    for result in results:
        plt.plot(result['depths'], result['errors'], 'o-', 
                label=result['params']['name'], linewidth=2, markersize=6)
    
    plt.xlabel('Distance (meters)')
    plt.ylabel('Relative Error (%) for 1px disparity error')
    plt.title('Depth Accuracy vs Distance')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.yscale('log')
    
    # 측정 범위 비교
    plt.subplot(1, 2, 2)
    ranges = []
    names = []
    
    for result in results:
        params = result['params']
        min_range = (params['focal_length'] * params['baseline']) / 80  # 최대 시차
        max_range = (params['focal_length'] * params['baseline']) / 1   # 최소 시차
        ranges.append([min_range, min(max_range, 20)])
        names.append(params['name'])
    
    y_pos = np.arange(len(names))
    for i, (name, range_val) in enumerate(zip(names, ranges)):
        plt.barh(i, range_val[1] - range_val[0], left=range_val[0], 
                alpha=0.7, label=name)
        plt.text(range_val[0] + (range_val[1] - range_val[0])/2, i, 
                f'{range_val[1] - range_val[0]:.1f}m', 
                ha='center', va='center', fontweight='bold')
    
    plt.yticks(y_pos, names)
    plt.xlabel('Distance (meters)')
    plt.title('Effective Measurement Range')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 분석 결과 출력
    print("\n📊 매개변수 영향 분석 결과:")
    print("\n🔍 초점거리 (Focal Length) 영향:")
    print("   - 클수록: 측정 범위 증가, 원거리 정확도 향상")
    print("   - 작을수록: 시야각 넓어짐, 근거리 측정 가능")
    
    print("\n📏 베이스라인 (Baseline) 영향:")
    print("   - 클수록: 전체적인 정확도 향상, 원거리 측정 가능")
    print("   - 작을수록: 시스템 소형화, 근거리 특화")
    
    print("\n⚖️ 트레이드오프:")
    print("   - 정확도 vs 측정 범위")
    print("   - 시스템 크기 vs 성능")
    print("   - 근거리 vs 원거리 최적화")
    
    print("\n💡 실무 선택 가이드:")
    print("   🏠 실내 로봇: 작은 베이스라인, 중간 초점거리")
    print("   🚗 자율주행: 큰 베이스라인, 긴 초점거리")
    print("   📱 모바일: 제약된 베이스라인, 최적화된 초점거리")
    print("   🏭 산업용: 정확도 우선, 큰 베이스라인")

# 카메라 매개변수 영향 분석 실행
analyze_camera_parameter_effects()

## 📏 실제 거리 측정 시스템 구현

In [ ]:
class DistanceMeasurementSystem:
    """
    실제 거리 측정을 위한 완전한 시스템
    """
    
    def __init__(self, depth_calculator):
        self.depth_calc = depth_calculator
        self.depth_map = None
        self.disparity_map = None
        
    def measure_distance_at_point(self, x, y, window_size=10):
        """
        특정 점에서의 거리 측정
        
        Parameters:
        - x, y: 측정하고자 하는 픽셀 좌표
        - window_size: 측정 윈도우 크기
        
        Returns:
        - distance: 측정된 거리 (미터)
        - confidence: 신뢰도 (0-1)
        """
        if self.depth_map is None:
            return None, 0
        
        h, w = self.depth_map.shape
        half_window = window_size // 2
        
        # 윈도우 범위 설정
        y_start = max(0, y - half_window)
        y_end = min(h, y + half_window + 1)
        x_start = max(0, x - half_window)
        x_end = min(w, x + half_window + 1)
        
        # 윈도우 내 유효한 깊이 값들
        window_depths = self.depth_map[y_start:y_end, x_start:x_end]
        valid_depths = window_depths[window_depths > 0]
        
        if len(valid_depths) == 0:
            return None, 0
        
        # 거리 계산 (중간값 사용으로 노이즈 감소)
        distance = np.median(valid_depths)
        
        # 신뢰도 계산 (일관성 기반)
        std_dev = np.std(valid_depths)
        coverage = len(valid_depths) / (window_size * window_size)
        
        # 신뢰도: 낮은 표준편차와 높은 커버리지일 때 높음
        confidence = coverage * np.exp(-std_dev / distance)
        confidence = min(1.0, confidence)
        
        return distance, confidence
    
    def measure_object_dimensions(self, roi):
        """
        관심 영역(ROI)의 객체 크기 측정
        
        Parameters:
        - roi: (x, y, width, height) 튜플
        
        Returns:
        - dimensions: 실제 크기 정보
        """
        x, y, w, h = roi
        
        if self.depth_map is None:
            return None
        
        # ROI 영역의 깊이 정보
        roi_depths = self.depth_map[y:y+h, x:x+w]
        valid_depths = roi_depths[roi_depths > 0]
        
        if len(valid_depths) == 0:
            return None
        
        avg_distance = np.mean(valid_depths)
        
        # 픽셀 크기를 실제 크기로 변환
        # 실제 크기 = (픽셀 크기 * 거리) / 초점거리
        real_width = (w * avg_distance) / self.depth_calc.focal_length
        real_height = (h * avg_distance) / self.depth_calc.focal_length
        
        return {
            'width_m': real_width,
            'height_m': real_height,
            'distance_m': avg_distance,
            'area_m2': real_width * real_height,
            'pixel_width': w,
            'pixel_height': h
        }
    
    def create_interactive_measurement_tool(self, left_image, disparity_map):
        """
        인터랙티브 거리 측정 도구 생성
        """
        self.disparity_map = disparity_map
        self.depth_map = self.depth_calc.disparity_to_depth(disparity_map)
        
        # 측정 시연
        measurement_points = [
            (200, 140, "Near Box Center"),
            (450, 150, "Middle Sphere"),
            (360, 380, "Far Block"),
            (150, 150, "Background Wall")
        ]
        
        plt.figure(figsize=(18, 12))
        
        # 원본 이미지에 측정점 표시
        plt.subplot(2, 3, 1)
        plt.imshow(left_image, cmap='gray')
        plt.title('Measurement Points\n(측정 지점들)', fontsize=14)
        
        for i, (x, y, label) in enumerate(measurement_points):
            plt.plot(x, y, 'ro', markersize=10)
            plt.text(x+10, y-10, f'{i+1}. {label}', color='red', 
                    bbox=dict(boxstyle="round", facecolor='white', alpha=0.8))
        
        plt.axis('off')
        
        # 깊이 맵
        plt.subplot(2, 3, 2)
        depth_display = np.where(self.depth_map > 0, self.depth_map, np.nan)
        plt.imshow(depth_display, cmap='viridis_r')
        plt.title('Depth Map\n(깊이 맵)', fontsize=14)
        plt.colorbar(label='Distance (m)')
        
        # 측정점들 다시 표시
        for i, (x, y, label) in enumerate(measurement_points):
            plt.plot(x, y, 'ro', markersize=8)
            plt.text(x+5, y-5, str(i+1), color='red', fontweight='bold')
        
        plt.axis('off')
        
        # 측정 결과 표시
        plt.subplot(2, 3, 3)
        plt.axis('off')
        
        results_text = "Distance Measurement Results\n"
        results_text += "=" * 40 + "\n\n"
        
        measurements = []
        for i, (x, y, label) in enumerate(measurement_points):
            distance, confidence = self.measure_distance_at_point(x, y)
            measurements.append((distance, confidence))
            
            if distance is not None:
                results_text += f"{i+1}. {label}:\n"
                results_text += f"   Distance: {distance:.2f} m\n"
                results_text += f"   Confidence: {confidence:.2f}\n\n"
            else:
                results_text += f"{i+1}. {label}:\n"
                results_text += f"   Distance: N/A\n"
                results_text += f"   Confidence: 0.00\n\n"
        
        plt.text(0.05, 0.95, results_text, transform=plt.gca().transAxes,
                verticalalignment='top', fontfamily='monospace', fontsize=10)
        
        # 신뢰도 히트맵
        plt.subplot(2, 3, 4)
        confidence_map = np.zeros_like(self.depth_map)
        
        h, w = self.depth_map.shape
        for y in range(0, h, 20):
            for x in range(0, w, 20):
                _, conf = self.measure_distance_at_point(x, y, window_size=6)
                confidence_map[y:y+20, x:x+20] = conf
        
        plt.imshow(confidence_map, cmap='RdYlGn')
        plt.title('Confidence Map\n(신뢰도 맵)', fontsize=14)
        plt.colorbar(label='Confidence')
        plt.axis('off')
        
        # ROI 크기 측정 예시
        plt.subplot(2, 3, 5)
        plt.imshow(left_image, cmap='gray')
        plt.title('Object Size Measurement\n(객체 크기 측정)', fontsize=14)
        
        # 예시 ROI들
        rois = [
            (150, 200, 100, 80, "Near Box"),
            (400, 100, 100, 100, "Middle Sphere")
        ]
        
        for x, y, w, h, label in rois:
            rect = Rectangle((x, y), w, h, linewidth=2, edgecolor='red', facecolor='none')
            plt.gca().add_patch(rect)
            
            # 크기 측정
            dimensions = self.measure_object_dimensions((x, y, w, h))
            if dimensions:
                plt.text(x, y-5, f"{label}\n{dimensions['width_m']:.2f}m x {dimensions['height_m']:.2f}m", 
                        color='red', fontweight='bold',
                        bbox=dict(boxstyle="round", facecolor='white', alpha=0.8))
        
        plt.axis('off')
        
        # 정확도 분석
        plt.subplot(2, 3, 6)
        
        actual_distances = [1.0, 2.0, 3.0, 6.0]  # 실제 거리
        measured_distances = [m[0] for m in measurements if m[0] is not None]
        confidences = [m[1] for m in measurements if m[0] is not None]
        
        if len(measured_distances) > 0:
            # 실제 vs 측정 거리
            valid_actual = actual_distances[:len(measured_distances)]
            
            plt.scatter(valid_actual, measured_distances, c=confidences, 
                       cmap='RdYlGn', s=100, alpha=0.8)
            plt.plot([0, max(valid_actual)], [0, max(valid_actual)], 'k--', alpha=0.5, label='Perfect')
            
            plt.xlabel('Actual Distance (m)')
            plt.ylabel('Measured Distance (m)')
            plt.title('Accuracy Analysis\n(정확도 분석)')
            plt.colorbar(label='Confidence')
            plt.legend()
            plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        print("\n📏 거리 측정 시스템 시연 완료!")
        print("\n🎯 측정 결과 분석:")
        for i, ((x, y, label), (distance, confidence)) in enumerate(zip(measurement_points, measurements)):
            if distance is not None:
                actual = [1.0, 2.0, 3.0, 6.0][i] if i < 4 else None
                if actual:
                    error = abs(distance - actual) / actual * 100
                    print(f"   {i+1}. {label}: {distance:.2f}m (오차: {error:.1f}%, 신뢰도: {confidence:.2f})")
                else:
                    print(f"   {i+1}. {label}: {distance:.2f}m (신뢰도: {confidence:.2f})")

# 거리 측정 시스템 생성 및 시연
measurement_system = DistanceMeasurementSystem(depth_calc)
measurement_system.create_interactive_measurement_tool(left_image, disparity_map)

## 🎓 실습 요약 및 응용 방향

In [ ]:
print("🎉 깊이 계산 및 3D 재구성 실습 완료!")
print("\n📚 오늘 학습한 핵심 내용:")
print("   1. 🧮 시차-깊이 변환: Z = (f × b) / d")
print("   2. 🌐 3D 점군 생성 및 시각화")
print("   3. 📏 실제 거리 측정 시스템 구현")
print("   4. 🔬 카메라 매개변수 영향 분석")
print("   5. 📊 측정 정확도 및 신뢰도 평가")

print("\n🔑 핵심 공식 및 개념:")
print("   📐 깊이 공식: depth = (focal_length × baseline) / disparity")
print("   🌐 3D 변환: X = (x-cx)×Z/f, Y = (y-cy)×Z/f, Z = depth")
print("   📏 실제 크기: size = (pixel_size × distance) / focal_length")
print("   🎯 신뢰도: coverage × exp(-std_dev/distance)")

print("\n⚖️ 정확도에 영향을 미치는 요인:")
print("   🔍 시차 크기: 클수록 정확 (가까운 거리)")
print("   📏 베이스라인: 클수록 정확")
print("   🔧 초점거리: 클수록 원거리 정확도 향상")
print("   🎨 영상 품질: 텍스처, 조명, 노이즈")
print("   ⚙️ 알고리즘: SGBM > BM, 후처리 중요")

print("\n🚀 실무 응용 분야:")
print("   🤖 로봇 내비게이션: 장애물 회피, 경로 계획")
print("   🚗 자율주행: 차간 거리, 보행자 감지")
print("   🏭 산업 자동화: 품질 검사, 크기 측정")
print("   📱 모바일 AR: 깊이 인식, 가상 객체 배치")
print("   🏥 의료 영상: 3D 재구성, 수술 보조")
print("   🎮 엔터테인먼트: 모션 캡처, VR/AR")

print("\n🎯 시스템 설계 가이드:")
print("   🎯 목적 정의: 측정 범위, 정확도 요구사항")
print("   📐 하드웨어 선택: 카메라, 렌즈, 베이스라인")
print("   🔧 소프트웨어 최적화: 알고리즘, 후처리")
print("   ✅ 캘리브레이션: 내부/외부 매개변수")
print("   📊 성능 평가: 정확도, 속도, 강건성")

print("\n🔮 다음 단계 학습 방향:")
print("   1. 🎥 실시간 스테레오비전 시스템")
print("   2. 🧠 딥러닝 기반 깊이 추정")
print("   3. 🔄 다시점 스테레오 (Multi-view Stereo)")
print("   4. 📡 LiDAR와 카메라 융합")
print("   5. 🎮 SLAM (Simultaneous Localization and Mapping)")

print("\n💡 실무 개발 팁:")
print("   🔧 프로토타이핑: 간단한 시스템부터 시작")
print("   📊 벤치마킹: 기존 솔루션과 비교")
print("   🎛️ 파라미터 튜닝: 환경에 맞게 최적화")
print("   🛡️ 에러 핸들링: 실패 케이스 대비")
print("   ⚡ 성능 최적화: GPU 활용, 병렬 처리")

print("\n🌟 스테레오비전의 미래:")
print("   🤖 AI와의 융합: 학습 기반 매칭")
print("   📱 모바일 통합: 실시간 AR/VR")
print("   🚗 자율주행 핵심 기술")
print("   🏭 Industry 4.0 핵심 센서")
print("   🌐 메타버스 공간 인식")

print("\n💪 수고하셨습니다!")
print("🎉 스테레오비전의 전체 파이프라인을 완전히 마스터하셨습니다!")
print("🚀 이제 실제 프로젝트에 적용해보세요!")